<a href="https://colab.research.google.com/github/Yurovskyy/TP3/blob/main/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import requests
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
# @title Carregando o dataset

# URL do arquivo CSV
url = 'https://github.com/Yurovskyy/TP3/raw/main/output.csv'

# Baixar o arquivo CSV da web
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Ler o conteúdo do arquivo CSV usando o Pandas
    df = pd.read_csv(url)

# Se a solicitação não for bem-sucedida
else:
    print('Falha ao baixar o arquivo CSV.')


In [3]:
# @title Exploração inicial do dataset

dimensoes = df.shape
print("Número de entradas(linhas):", dimensoes[0])
print("Número de colunas(variáveis):", dimensoes[1])

# identificar o tipo de cada variável
display(df.dtypes)

# ver as 5 primeiras entradas do conjunto de treino
display(df.head())

Número de entradas(linhas): 4944
Número de colunas(variáveis): 14


,0
Index,float64
firm,int64
BE,float64
ME,float64
ME.lag1,float64
r_i,float64
r.3.0,float64
r.12.4,float64
AC,float64
CA,float64


,Index,firm,BE,ME,ME.lag1,r_i,r.3.0,r.12.4,AC,CA,DR,OFC,ECI,industry
0,2020.75,1,1.179763e+09,1.217182e+09,1.200622e+09,0.089471,-0.013699,-0.618101,0.500000,0.694444,0.750000,0.433333,0.805556,Outros
1,2020.50,1,1.262906e+09,1.064582e+09,1.217182e+09,-0.013699,0.142185,-0.471392,0.500000,0.611111,0.750000,0.400000,0.833333,Outros
2,2022.00,1,1.205650e+09,1.354615e+09,1.761918e+09,0.181542,0.259363,0.039740,0.527778,0.722222,0.750000,0.666667,0.833333,Outros
3,2021.25,1,1.190537e+09,1.312840e+09,1.064523e+09,0.267479,-0.209721,0.217958,0.500000,0.694444,0.750000,0.433333,0.805556,Outros
4,2023.00,1,1.072554e+09,2.471660e+09,2.544702e+09,0.042154,0.029124,0.337521,0.527778,0.888889,0.958333,0.766667,0.722222,Outros


In [4]:
# @title Valores faltantes
# @markdown Como eu disse, não temos valores faltantes!

# ver a porcentagem valores faltantes
display((df.isnull().sum() / df.shape[0]).sort_values(ascending=False))

,0
Index,0.0
firm,0.0
BE,0.0
ME,0.0
ME.lag1,0.0
r_i,0.0
r.3.0,0.0
r.12.4,0.0
AC,0.0
CA,0.0


In [5]:
# @title Descrição dos valores categóricos

display(df.describe(include=['O']))

,industry
count,4944
unique,20
top,Outros
freq,1145


In [7]:
fig = px.scatter(df,
                 x='ME',
                 y='r.12.4',
                 hover_data=["industry","firm"],
                 title="Market Value por Retorno Anual")

fig.show()

In [8]:
fig = px.scatter(df,
                 x='industry',
                 y='r.12.4',
                 hover_data=["ME","firm"],
                 title="Tipo de industria por Retorno Anual")

fig.show()

In [10]:
fig = px.scatter(df,
                 x='r.12.4',
                 y='CA',
                 hover_data=["industry","firm","ME"],
                 title="CA por Retorno Anual")

fig.show()

In [11]:
fig = px.scatter(df,
                 x='r.12.4',
                 y='AC',
                 hover_data=["industry","firm","ME"],
                 title="AC por Retorno Anual")

fig.show()

In [12]:
fig = px.scatter(df,
                 x='r.12.4',
                 y='DR',
                 hover_data=["industry","firm","ME"],
                 title="DR por Retorno Anual")

fig.show()

In [13]:
fig = px.scatter(df,
                 x='r.12.4',
                 y='OFC',
                 hover_data=["industry","firm","ME"],
                 title="OFC por Retorno Anual")

fig.show()

In [14]:
fig = px.scatter(df,
                 x='r.12.4',
                 y='ECI',
                 hover_data=["industry","firm","ME"],
                 title="ECI por Retorno Anual")

fig.show()

In [20]:
fig = px.histogram(df, x='r.12.4', title='Histograma dos valores')
fig.show()

In [16]:
display(df.describe())

,Index,firm,BE,ME,ME.lag1,r_i,r.3.0,r.12.4,AC,CA,DR,OFC,ECI
count,4944.000000,4944.000000,4.944000e+03,4.944000e+03,4.944000e+03,4944.000000,4944.000000,4944.000000,4944.000000,4944.000000,4944.000000,4944.000000,4944.000000
mean,2021.700496,418.813714,9.244580e+09,1.647379e+10,1.655128e+10,0.003556,0.003051,0.017635,0.528727,0.781031,0.845292,0.710720,0.764979
std,1.326880,245.360361,2.990092e+10,4.501468e+10,4.522360e+10,0.275630,0.276638,0.441591,0.148566,0.130017,0.135652,0.165161,0.139815
min,2019.000000,1.000000,5.600000e+04,1.597143e+06,1.597557e+06,-2.266958,-1.392217,-1.960695,0.194444,0.361111,0.333333,0.233333,0.305556
25%,2020.500000,209.000000,7.253755e+08,8.281819e+08,8.229385e+08,-0.121592,-0.126219,-0.211830,0.416667,0.694444,0.791667,0.600000,0.694444
50%,2021.750000,405.000000,1.794868e+09,3.431592e+09,3.423414e+09,0.012751,0.009830,0.021291,0.527778,0.805556,0.875000,0.733333,0.777778
75%,2022.750000,622.000000,6.121884e+09,1.161053e+10,1.183215e+10,0.148643,0.151809,0.254705,0.638889,0.888889,0.958333,0.833333,0.861111
max,2023.750000,860.000000,4.341940e+11,5.705210e+11,5.705210e+11,1.516664,1.516664,5.134580,0.833333,1.000000,1.000000,1.000000,1.000000


In [28]:
df = df.drop(columns=["industry"])

KeyError: "['industry'] not found in axis"

In [30]:
scaler = StandardScaler()
values = scaler.fit_transform(df[['r.12.4',"AC","CA","DR","OFC","ECI"]])

kmeans = KMeans(n_clusters=5, random_state=42)
df['Cluster'] = kmeans.fit_predict(values)

fig = px.scatter(df, x='Index', y='r.12.4', color='Cluster', hover_data=["Index","firm","BE","ME","r.12.4","AC","CA","DR","OFC","ECI"], title='Clusterização')
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [27]:

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(df)

fig = px.scatter(X_reduced, x=0, y=1, color=df["Cluster"].astype(str),hover_data=["Index","firm","BE","ME","r.12.4","AC","CA","DR","OFC","ECI"], title="Clusterização com KMeans")
fig.show()